Fix the dataset (woops)

In [1]:
import os
import re

# Define the regex pattern for the date and time timestamp
timestamp_pattern = re.compile(r'_\d{4}-\d{2}-\d{2}-\d{6}-\d{4}_')

def rename_images_in_folder(folder_path):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.png'):  # You can add other extensions if needed
                old_path = os.path.join(root, file)
                # Find the timestamp pattern and remove it
                new_file = re.sub(timestamp_pattern, '_', file)
                new_path = os.path.join(root, new_file)
                os.rename(old_path, new_path)
                print(f'Renamed: {old_path} -> {new_path}')

# Replace 'your_folder_path' with the path to your folder
folder_path = 'drive/MyDrive/Colab Notebooks/Zebrafish Frames and Annotations/images'
rename_images_in_folder(folder_path)

Create a train/test/val split in the dataset

In [9]:
import os
import random
import shutil
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor

def train_test_val_split_image_annotation(image_dir, annotation_dir, output_dir, train_size=0.7, val_size=0.15, test_size=0.15, random_state=None):
    """
    Splits image and annotation files into train, validation, and test sets and writes them to new directories.

    Parameters:
    image_dir (str): Directory containing image files.
    annotation_dir (str): Directory containing annotation files.
    output_dir (str): Base directory where images and labels directories will be created.
    train_size (float): Proportion of the dataset to include in the train split.
    val_size (float): Proportion of the dataset to include in the validation split.
    test_size (float): Proportion of the dataset to include in the test split.
    random_state (int): Controls the shuffling applied to the data before applying the split.
    """
    # Get list of image files
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]

    # Ensure the annotations exist
    image_annotation_pairs = []
    for img in image_files:
        img_path = os.path.join(image_dir, img)
        ann_path = os.path.join(annotation_dir, img.replace('.png', '.txt'))
        if os.path.isfile(ann_path):
            image_annotation_pairs.append((img_path, ann_path))
        else:
            print(f"Skipping {img} as corresponding annotation {img.replace('.png', '.txt')} does not exist.")

    # Shuffle data
    if random_state is not None:
        random.seed(random_state)
    random.shuffle(image_annotation_pairs)

    # Split into train+val and test sets
    train_val_size = train_size + val_size
    train_val_split, test_split = train_test_split(image_annotation_pairs, test_size=test_size, random_state=random_state)

    # Calculate the proportion of train/val split
    val_adjusted_size = val_size / train_val_size

    # Split the train+val set into train and val sets
    train_split, val_split = train_test_split(train_val_split, test_size=val_adjusted_size, random_state=random_state)

    # Helper function to copy files
    def copy_files(file_pairs, subset_name):
        image_output_dir = os.path.join(output_dir, 'images', subset_name)
        annotation_output_dir = os.path.join(output_dir, 'labels', subset_name)
        os.makedirs(image_output_dir, exist_ok=True)
        os.makedirs(annotation_output_dir, exist_ok=True)

        def copy_pair(pair):
            img_path, ann_path = pair
            shutil.copy(img_path, image_output_dir)
            shutil.copy(ann_path, annotation_output_dir)

        with ThreadPoolExecutor() as executor:
            executor.map(copy_pair, file_pairs)

    # Copy files to train, val, and test directories
    copy_files(train_split, 'train')
    copy_files(val_split, 'val')
    copy_files(test_split, 'test')

    print("Train files:", len(train_split))
    print("Validation files:", len(val_split))
    print("Test files:", len(test_split))


# Example usage
if __name__ == "__main__":
    # Directories containing images and annotations
    image_dir = '/home/jovyan/work/datasets/Zebrafish Embryos Dataset V2 Early Frames/images'
    annotation_dir = '/home/jovyan/work/datasets/Zebrafish Embryos Dataset V2 Early Frames/labels'
    output_dir = '/home/jovyan/work/datasets/Zebrafish Embryos Dataset V2 Early Frames/data'  # Base directory where images and labels directories will be created

    # Splitting the data and copying to new directories
    train_test_val_split_image_annotation(image_dir, annotation_dir, output_dir, train_size=0.8, val_size=0.10, test_size=0.10, random_state=42)


Skipping Tyler Double KO Touch Responses_Double KO 15_frame_298.png as corresponding annotation Tyler Double KO Touch Responses_Double KO 15_frame_298.txt does not exist.
Train files: 16010
Validation files: 2002
Test files: 2002


In [10]:
!ls '/home/jovyan/work/datasets/Zebrafish Embryos Dataset V2 Early Frames/data/images'
!ls '/home/jovyan/work/datasets/Zebrafish Embryos Dataset V2 Early Frames/data/labels'

test  train  val
test  train  val


Now Install Ultralytics and set up the YOLOv8 model for training

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.7/793.7 kB 9.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 10.7 MB/s eta 0:00:0000:0100:01


Change YOLO's default dataset directory

In [3]:
!yolo settings

💡 Learn about settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings
Printing '/home/jovyan/.config/Ultralytics/settings.yaml'

settings_version: 0.0.4
datasets_dir: /home/jovyan/work/datasets
weights_dir: weights
runs_dir: runs
uuid: af86e989f891cbd0cd78de981596cc553f47f4a708b5ec210405ae2024cb9aec
sync: true
api_key: ''
openai_api_key: ''
clearml: true
comet: true
dvc: true
hub: true
mlflow: true
neptune: true
raytune: true
tensorboard: true
wandb: true



Download and run the model

In [2]:
ROOT_DIR = '/home/jovyan/work/datasets/Zebrafish Embryos Dataset V2 Early Frames'

### Notes for training YOLOv8

from [https://github.com/ultralytics/ultralytics/issues/4106](https://github.com/ultralytics/ultralytics/issues/4106)

@FatemaD1577 It really depends on a lot of parameters to answer this question. Here are some examples:

It depends on the model you will be using, for example yolov8n is much smaller model that yolov8x and will consume far less GB of GPU or RAM space for the training.
Also, the size of the training images which is specified in imgsz parameter is very crucial, larger image size for training images will consume more GB while training.
If your GPU is any new nvidia model and supports Mixed Precision then less space will be needed for the training.
Batch size is also a crucial parameter which plays a significant role in the training procedure and higher batch_size consume more memory while training.
Typically if you have a 4GB GPU without support for Mixed Precision, you will need to train using imgsz = 320 which i think is the lowest resolution accepted by the model and small batch_size like 1,2,4.

If you have 8GB of GPU or more, then you have nothing to worry you can train easy.

On CPU your training is going to be slower but if you have 12GB or more of RAM you can train.

In [3]:
import os
from ultralytics import YOLO


#Load a model
model = YOLO("yolov8n.pt")

#Use the model
#results = model.train(data=os.path.join(ROOT_DIR, "zebratrack.yaml"), epochs=1)
# this determines optimal batch size, but training still crashes. batch size was calculated to be 42.
#results = model.train(data=os.path.join(ROOT_DIR, "zebratrack.yaml"), epochs=1, batch=-1)
# we try batch size 1 - smallest possible - to see if we can stop crashing. (We crash at 10+...)
results = model.train(data=os.path.join(ROOT_DIR, "zebratrack.yaml"), epochs=400, batch=8, workers=4, patience=10)


Ultralytics YOLOv8.2.48 🚀 Python-3.11.8 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/jovyan/work/datasets/Zebrafish Embryos Dataset V2 Early Frames/zebratrack.yaml, epochs=400, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=train26, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fa

100%|████████████████████████████████████████| 755k/755k [00:00<00:00, 9.86MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 225 layers, 3011043 parameters, 3011027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jovyan/work/datasets/Zebrafish Embryos Dataset V2 Early Fr
val: Scanning /home/jovyan/work/datasets/Zebrafish Embryos Dataset V2 Early Fram


...

    347/400       1.2G     0.8054     0.4138     0.8434          5        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all       2002       2002      0.992      0.987      0.994      0.789


EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 337, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

347 epochs completed in 22.235 hours.
Optimizer stripped from runs/detect/train26/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train26/weights/best.pt, 6.3MB

Validating runs/detect/train26/weights/best.pt...
Ultralytics YOLOv8.2.48 🚀 Python-3.11.8 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all       2002       2002      0.992      0.986      0.994      0.789
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train26
